In [ ]:
import cv2
import numpy as np
import pickle
import os

block_size = 8
num_clusters = 5  # Number of clusters per block
num_bits_for_color = 4

def quantize_colors(block, num_bits=4):
    # Quantize color information by reducing the number of bits
    return (block >> (8 - num_bits)) << (8 - num_bits)

def decompress_and_save_video(frames_data, output_file, compressed_data):
    # Create VideoWriter object to save the decompressed video
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out_video = cv2.VideoWriter(output_file, fourcc, 30.0, (1280, 720))
    # Decompress and add to the overall video
    reconstructed_frames = []
    for frame_data in compressed_data:
        reconstructed_frame = np.zeros((720, 1280, 3), dtype=np.uint8)

        for i, block_info in enumerate(frame_data):
            if 'motion_vector' in block_info:
                # Update the color cluster centers based on the motion vector
                motion_vector = np.array(block_info['motion_vector'])
                if initial_centers is not None:
                    initial_centers += motion_vector
            else:
                # Use the predicted or initial cluster centers to reconstruct the block
                centers = np.array(block_info['cluster_centers'])
                labels = np.array(block_info['labels'])

                reshaped_block = centers[labels.flatten()].reshape((block_size, block_size, 3))

                # Calculate block coordinates
                y, x = divmod(i, int(1280 / block_size))
                y *= block_size
                x *= block_size

                # Ensure the indices stay within the frame boundaries
                y_end, x_end = y + block_size, x + block_size
                y_end = min(y_end, 720)
                x_end = min(x_end, 1280)

                reconstructed_frame[y:y_end, x:x_end] = reshaped_block[:y_end - y, :x_end - x].astype(np.uint8)

        # Write the decompressed frame to the video file
        out_video.write(reconstructed_frame)

    # Release the video file
    out_video.release()

def combine_videos(output_file_prefix, part_counter):
    # Create VideoWriter object to save the combined video
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    combined_output_file = "combined_output.avi"
    out_combined_video = cv2.VideoWriter(combined_output_file, fourcc, 30.0, (1280, 720))

    # Iterate through all the decompressed video parts and append frames to the combined video
    for i in range(1, part_counter):
        input_file = f"{output_file_prefix}{i}.avi"
        cap = cv2.VideoCapture(input_file)

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            out_combined_video.write(frame)

        cap.release()

    # Release the combined video file
    out_combined_video.release()

    print(f"\nCombined video saved as: {combined_output_file}")

def main():
    input_file = "/kaggle/input/ml-projectdatet/VideoCompressionDataset/AlitaBattleAngel.mkv"
    output_file_prefix = "output_decompressed_video_part_"
    data_file = "compressed_data2.pkl"

    cap = cv2.VideoCapture(input_file)
    total_frames = int(min(cap.get(cv2.CAP_PROP_FRAME_COUNT), 800))  # Process the first 60 frames

    frames_data = []
    part_counter = 1

    print("Compressing video and decompressing every 60 frames...")

    current_frame = 0
    while current_frame<total_frames:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (1280, 720))

        frame_data = []

        # Divide frame into blocks
        initial_centers = None
        height, width = frame.shape[:2]
        for y in range(0, height, block_size):
            for x in range(0, width, block_size):
                block = frame[y:y+block_size, x:x+block_size]
#                 quantized_block = quantize_colors(block, num_bits=num_bits_for_color)

                # Perform k-means clustering on the block
                reshaped_block = np.float32(block.reshape((-1, 3)))
                criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 25, 0.2)
                _, labels, centers = cv2.kmeans(reshaped_block, num_clusters, None, criteria, 2, cv2.KMEANS_RANDOM_CENTERS)

                # Save the information for compression
                block_info = {
                    'cluster_centers': centers.tolist(),
                    'labels': labels.flatten().tolist()
                }

                frame_data.append(block_info)

        frames_data.append(frame_data)

        # Track motion of color clusters
        if initial_centers is None:
            initial_centers = centers
        else:
            motion_vector = initial_centers - centers
            initial_centers = centers

            # Save the motion vector for compression
            frame_data.append({'motion_vector': motion_vector.tolist()})

        current_frame += 1
        print(f"Compression Progress: {current_frame}/{total_frames}", end='\r')

        # Decompress and save individual video for the current 60 frames
        if current_frame % 30 == 0 or current_frame == total_frames:
            # Save frames data to the pickle file
            with open(data_file, 'wb') as file:
                pickle.dump(frames_data, file)

            # Decompress and save individual video for the current 60 frames
            output_file = output_file_prefix + str(part_counter) + ".avi"
            decompress_and_save_video(frames_data, output_file, frames_data)
            print(f"Decompression completed for frames {current_frame - 29} to {current_frame}.")

            # Clear frames_data for the next batch of frames
            frames_data = []
            part_counter += 1

    cap.release()

    # Combine all decompressed output videos after processing all frames
    combine_videos(output_file_prefix, part_counter)

    print("\nCompression and Decompression completed.")

if _name_ == "_main_":
    main()